In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip uninstall nltk -y
!pip install texthero

Found existing installation: nltk 3.2.4
Uninstalling nltk-3.2.4:
  Successfully uninstalled nltk-3.2.4
     |████████████████████████████████| 1.4 MB 1.3 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434678 sha256=570b5627f6948652b019d7245de3d631ec0a212eaa5ee36a663735cdbe525a40
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.5 which is incompatible.


In [ ]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import gc
from tqdm import tqdm
import time
import nltk
import random

import texthero as hero
from texthero import preprocessing

NFOLDS=5
SEED=42
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

set_seed(SEED)
INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

データの読み込み

In [ ]:
train_df = pd.read_json(os.path.join(INPUT_DIR, 'train_data.json'), lines=True)
test_df = pd.read_json(os.path.join(INPUT_DIR, 'test_data.json'), lines=True)
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
print(train_df.shape, test_df.shape, all_df.shape)

all_df.head()

(851524, 16) (59084, 15) (910608, 16)


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[{'version': 'v1', 'created': 'Wed, 10 Feb 199...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[{'version': 'v1', 'created': 'Tue, 3 Jul 2018...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[{'version': 'v1', 'created': 'Wed, 15 May 201...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN


In [ ]:
del train_df
del test_df
gc.collect()

20

In [ ]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [ ]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))

# ベクトル化
vec_tfidf = TfidfVectorizer(min_df=20)
abstract_df = vec_tfidf.fit_transform(all_title_abstract['clean_abstract'].values)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=SEED)
kmeans_abstract_df = pd.DataFrame(kmeans.fit_transform(abstract_df))
kmeans_abstract_df.columns = ['kmeans_tfidf_abstract_' + str(col) for col in kmeans_abstract_df.columns]
kmeans_abstract_df = pd.concat([all_df['id'], kmeans_abstract_df], axis=1)

In [ ]:
kmeans_abstract_df.to_feather(os.path.join(INPUT_DIR, 'kmeans_tfidf_abstract_df.feather'))